# Solving Quadratically Constrained Quadratic Programming Models


In [1]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

## Quadratically constrained quadratic programming
Quadratically constrained quadratic programming concerns the optimization of a quadratic objective function subject to quadratic constraints and linear constraints, i.e., the problem:

\begin{align*}
\mbox{minimize} \;\;& \sum\limits_{i=1}^{N} c_{0i} x_{i} + \dfrac{1}{2} \sum\limits_{i=1}^{N} \sum\limits_{j=1}^{N} q_{0ij} x_{i} x_{j} \\
\mbox{subject to:} \;\;& \sum\limits_{i=1}^{N} c_{ni} x_{i} + \sum\limits_{i=1}^{N} \sum\limits_{j=1}^{N} q_{nij} x_{i} x_{j} \leq d_{n} \;\;\;\; n = 1, 2, \dots, P\\
 \;\;& \sum\limits_{i=1}^{N} a_{mi} x_{i} \leq b_{m} \;\;\;\; m = 1, 2, \dots, M\\
\;\;& l_{i} \leq x_{i} \leq u_{i} \;\;\;\; i = 1, 2, \dots, N.
\end{align*}

The same problem can be converted into matrix-vector notation as follows:

\begin{align*}
\mbox{minimize} \;\;& \boldsymbol{c}_{0}^{\top} \boldsymbol{x} + \dfrac{1}{2} \boldsymbol{x}^{\top} \mathbf{Q}_{0} \boldsymbol{x} \\
\mbox{subject to:} \;\;& \boldsymbol{c}_{n}^{\top} \boldsymbol{x} + \boldsymbol{x}^{\top} \mathbf{Q}_{n} \boldsymbol{x} \leq d_{n} \;\;\;\; n = 1, 2, \dots, P\\
 \;\;& \mathbf{A} \boldsymbol{x} \leq \boldsymbol{b} \\
\;\;& \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}.
\end{align*}

### Example problem
\begin{align*}
\mbox{maximize} \;\;& x_{1} + 2 x_{2} + 3 x_{3} - \dfrac{1}{2}\left(33x_{1}^{2} + 22x_{2}^{2} + 11x_{3}^{2} - 12 x_{1} x_{2} - 23 x_{2} x_{3} \right) \\
\mbox{subject to:}\;\;& x_{1}^{2} + x_{2}^{2} + x_{3}^{2} \leq 1 \\
\;\;& -x_{1} +x_{2} + x_{3} \leq 20 \\
\;\;& x_{1} - 3x_{2} + x_{3} \leq 30 \\
\;\;&  0 \leq x_{1} \leq 40\\
\;\;&  0 \leq x_{2} \leq 40\\
\;\;&  0 \leq x_{3} \leq 40.
\end{align*}

\begin{align*}
\mbox{maximize} \;\;& \begin{bmatrix}1 & 2 & 3\end{bmatrix} \begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} +  \dfrac{1}{2}\begin{bmatrix} x_{1} & x_{2} & x_{3}\end{bmatrix} \begin{bmatrix} -33 & 6 & 0\\ 6 & -22 & 11.5\\ 0 & 11.5 & -11\end{bmatrix} \begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} \\
\mbox{subject to:}\;\;& \begin{bmatrix}0 & 0 & 0\end{bmatrix} \begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} + \begin{bmatrix} x_{1} & x_{2} & x_{3}\end{bmatrix} \begin{bmatrix} 1 & 0 & 0\\ 0 & 1 & 0\\ 0 & 0 & 1\end{bmatrix} \begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} \leq 1 \\
\;\;& \begin{bmatrix} -1 & 1 & 1\\ 1 & -3 & 1\end{bmatrix}\begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} \leq \begin{bmatrix} 20 \\30 \end{bmatrix} \\
\;\;&  \begin{bmatrix} 0\\ 0\\ 0\end{bmatrix} \leq \begin{bmatrix} x_{1}\\ x_{2}\\ x_{3}\end{bmatrix} \leq \begin{bmatrix} 40\\ 40\\ 40\end{bmatrix}.
\end{align*}

In [2]:
direction = "maximize"
c0 = np.array([1, 2, 3])
Q0 = sp.csr_matrix(np.array([[-33, +6, 0],
                             [+6, -22, +11.5],
                             [0, +11.5, -11]]))

c1 = np.array([0, 0, 0])
Q1 = sp.csr_matrix(np.array([[1, 0, 0],
                             [0, 1, 0],
                             [0, 0, 1]]))
sense1 = "L"
d1 = 1.0

A = sp.csr_matrix(np.array([[-1, +1, +1],
                            [+1, -3, +1]]))
senses = np.array(["L", "L"])
b = np.array([20, 30])

l = np.array([0, 0, 0])
u = np.array([40, 40, 40])

In [3]:
# create an empty optimization problem
prob = cp.Cplex()

# add decision variables to the problem including their linear coefficients in objective and ranges
prob.variables.add(obj = c0.tolist(), lb = l.tolist(), ub = u.tolist())

# add quadratic coefficients in objective
row_indices, col_indices = Q0.nonzero()
prob.objective.set_quadratic_coefficients(zip(row_indices.tolist(), col_indices.tolist(), Q0.data.tolist()))

# define problem type
if direction == "maximize":
    prob.objective.set_sense(prob.objective.sense.maximize)
else:
    prob.objective.set_sense(prob.objective.sense.minimize)

# add constraints to the problem including their directions and right-hand side values
prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

# add coefficients for each constraint
row_indices, col_indices = A.nonzero()
prob.linear_constraints.set_coefficients(zip(row_indices.tolist(), col_indices.tolist(), A.data.tolist()))

# add quadratic constraints to the problem including their directions and right-hand side values
row_indices, col_indices = Q1.nonzero()
prob.quadratic_constraints.add(quad_expr = cp.SparseTriple(row_indices.tolist(), 
                                                           col_indices.tolist(), 
                                                           Q1.data.tolist()),
                               sense = sense1, rhs = d1)

print(prob.write_as_string())

# solve the problem
prob.solve()

# check the solution status
print(prob.solution.get_status())
print(prob.solution.status[prob.solution.get_status()])

# get the solution
x_star = prob.solution.get_values()
obj_star = prob.solution.get_objective_value()

print(x_star, obj_star)

Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 obj1: x1 + 2 x2 + 3 x3 + [ - 33 x1 ^2 + 12 x1 * x2 - 22 x2 ^2 + 23 x2 * x3
       - 11 x3 ^2 ] / 2
Subject To
 c1: - x1 + x2 + x3 <= 20
 c2: x1 - 3 x2 + x3 <= 30
 q1: [ x1 ^2 + x2 ^2 + x3 ^2 ] <= 1
Bounds
 0 <= x1 <= 40
 0 <= x2 <= 40
 0 <= x3 <= 40
End

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Aggregator did 1 substitutions.
Reduced QCP has 14 rows, 9 columns, and 24 nonzeros.
Reduced QCP has 2 quadratic constraints.
Presolve time = 0.00 sec. (0.01 ticks)
Parallel mode: using up to 2 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 73
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.00 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 2
  Rows in Factor            = 14
  Integer space required    = 14
  Total non-zeros in factor = 105
  Total FP ops to factor    =